In [ ]:
import torch
from PIL import Image
import numpy as np
import cv2

from invokeai.backend.vto_workflow.overlay_pattern import generate_dress_mask
from invokeai.backend.vto_workflow.extract_channel import extract_channel, ImageChannel


In [ ]:
 # Load the model image.
model_image = Image.open("/home/ryan/src/InvokeAI/invokeai/backend/vto_workflow/dress.jpeg")

# Load the pattern image.
pattern_image = Image.open("/home/ryan/src/InvokeAI/invokeai/backend/vto_workflow/pattern1.jpg")

In [ ]:
mask = generate_dress_mask(model_image)

In [ ]:
# Visualize mask
model_image_np = np.array(model_image)
masked_model_image = (model_image_np * np.expand_dims(mask, -1).astype(np.float32)).astype(np.uint8)
mask_image = Image.fromarray(masked_model_image)
mask_image

In [ ]:
shadows = extract_channel(np.array(model_image), ImageChannel.LAB_L)

In [ ]:
# Visualize masked shadows
masked_shadows = (shadows * mask).astype(np.uint8)
masked_shadows_image = Image.fromarray(masked_shadows)
masked_shadows_image